In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import datetime as dt

In [2]:
df_train = pd.read_csv('/home/skang/Documents/kaggle/bike_sharing/input/train.csv')
df_train['windspeed'].fillna((df_train['windspeed'].median()), inplace=True)
df_train['humidity'].fillna((df_train.groupby('season')['humidity'].transform('median')), inplace=True)
mean_temp = (df_train.iloc[700]['temp'] + df_train.iloc[702]['temp']) / 2
mean_atemp = (df_train.iloc[700]['atemp'] + df_train.iloc[702]['atemp']) / 2
df_train['temp'].fillna((mean_temp), inplace=True)
df_train['atemp'].fillna((mean_atemp), inplace=True)
df_train = df_train.drop('datetime', axis=1)
df_train.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
X = df_train.iloc[:,:-1]
y = df_train.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2022)

In [4]:
res_1 = y_train - np.mean(y_train) # xgboost의 기본 첫 학습값은 회귀->평균, 분류->다수인 클래스 레이블

In [5]:
tree_1 = DecisionTreeRegressor(max_depth=2, random_state=2022)
tree_1.fit(X_train, res_1)
pred_1 = tree_1.predict(X_train)
res_2 = y_train - pred_1

In [6]:
tree_2  = DecisionTreeRegressor(max_depth=2, random_state=2022)
tree_2.fit(X_train, res_2)
pred_2 = tree_2.predict(X_train)
res_3 = y_train - (pred_1 + pred_2)

In [7]:
tree_3 = DecisionTreeRegressor(max_depth=2, random_state=2022)
tree_3.fit(X_train, res_3)

from sklearn.metrics import mean_squared_error

pred_all = tree_1.predict(X_test) + tree_2.predict(X_test) + tree_3.predict(X_test)
mean_squared_error(y_test, pred_all, squared=False)

46.600805543061114